In [ ]:
import serial
import time
import re

# --- CONFIGURAÇÕES DO MANUAL SARTORIUS CCE ---
PORTA = 'COM8'       # Ajuste para sua porta
BAUDRATE = 1200      # Padrão de fábrica (verifique no menu da balança se foi alterado para 9600)
BYTESIZE = serial.SEVENBITS  # IMPORTANTE: Balanças CCE usam 7 bits por padrão
PARITY = serial.PARITY_ODD   # IMPORTANTE: Paridade Ímpar
STOPBITS = serial.STOPBITS_ONE
TIMEOUT = 1

def conectar_sartorius():
    try:
        ser = serial.Serial(
            port=PORTA,
            baudrate=BAUDRATE,
            bytesize=BYTESIZE,
            parity=PARITY,
            stopbits=STOPBITS,
            timeout=TIMEOUT,
            rtscts=True  # O manual cita "Hardware Handshake" como padrão
        )
        print(f"Conectado à Sartorius CCE na {PORTA}")
        return ser
    except serial.SerialException as e:
        print(f"Erro de conexão: {e}")
        return None

def ler_peso(ser):
    """
    Lê o dado da balança. A Sartorius envia dados em formato de 16 ou 22 caracteres.
    Exemplo de retorno: "   +  150.05 g  "
    """
    if ser.in_waiting > 0:
        raw_data = ser.readline()
        try:
            # Decodifica e limpa espaços extras
            texto = raw_data.decode('ascii', errors='ignore').strip()
            print(f"Raw: {texto}")
            
            # Regex para capturar o número com sinal e decimais
            # Procura por algo como "+ 10.5" ou "- 2.0"
            match = re.search(r"[-+]?\s*\d+\.\d+", texto)
            if match:
                # Remove espaços dentro do número (ex: "+ 10" vira "+10")
                peso_str = match.group().replace(" ", "")
                return float(peso_str)
        except:
            pass
    return None

def enviar_comando(ser, comando):
    """
    Envia comandos ESC para a balança (Protocolo SBI).
    O manual diz que comandos iniciam com ESC (ASCII 27) e terminam com _ (ASCII 95) + CRLF
    """
    comandos = {
        'TARAR': b'\x1bf4_',       # Esc + f4 + _ (Tarar)
        'ZERO': b'\x1bf3_',        # Esc + f3 + _ (Zerar)
        'PRINT': b'\x1bP',         # Esc + P (Solicitar peso se não estiver automático)
        'RESTART': b'\x1bS_'       # Restart
    }
    
    if comando in comandos:
        cmd_bytes = comandos[comando] + b'\r\n'
        ser.write(cmd_bytes)
        print(f"Comando {comando} enviado.")
        time.sleep(0.5) # Tempo para processar

# --- LOOP PRINCIPAL ---
if __name__ == "__main__":
    balanca = conectar_sartorius()
    
    if balanca:
        try:
            # Opcional: Envia um comando de Print para forçar a saída de dados
            # enviar_comando(balanca, 'PRINT') 
            
            while True:
                peso = ler_peso(balanca)
                if peso is not None:
                    print(f"PESO LIDO: {peso} g")
                
                time.sleep(0.1)

        except KeyboardInterrupt:
            print("Parando...")
            balanca.close()

: 